### Self consistent field

Now we have everything we need to run the SCF cycle! We'll combine here everything we have built so far to recap.

We also need to define some parameters for our SCF loop: a threshold for convergence and the maximum number of iterations, so that our loop has a definite end point. We'll print out the energy and the matrix norm of the difference between the new and old density matrices.

In [47]:
from pyscf import gto, scf, mp
from ase.units import Bohr
import numpy as np

# Define the system in PySCF and get all the molecular integrals
m = gto.Mole()
m.build(atom=f"H 0 0 0; H {1.4*Bohr} 0 0", basis="3-21g", spin=0, charge=0)
Tmat = m.intor("int1e_kin") # Integral matrix for kinetic energy
Smat = m.intor("int1e_ovlp") # Integral matrix for overlap integrals
Vmat = m.intor("int1e_nuc") # Integral matrix for electron-nucleus integrals 
Vee = m.intor("int2e") # Integral matrix for electron-electron integrals
nelec = m.nelectron # Number of electrons

In [55]:
# Orthogonalization matrix
s, U = np.linalg.eigh(Smat)
X = U / s**0.5

# Core Hamiltonian initial guess
Hcore = Tmat + Vmat
Hcore_ = np.dot(X.T, Hcore.dot(X)) # Transform from AO to MO basis
mo_eigs, mo_vecs = np.linalg.eigh(Hcore_) # Diagonalize the Hcore in MO basis
# Construct the density matrix from the initial guess
Cvec = np.dot(X, mo_vecs) # Transform the MO coefficients from MO to AO basis
Dmat = np.zeros((Smat.shape)) # Initialize the density matrix
for a in range(nelec//2): # Compute the density matrix in the AO basis. Since we use restricted HF, all spins are paired and only nelec/2 orbitals are considered
    Dmat += np.einsum('i,j->ij', Cvec[:, a], Cvec[:, a].T)

What exactly is the electronic energy, $E_0$, of the Hartree-Fock system? One might think, that since we have $N$ electrons, the total energy would be the sum of the eigenenergies for each electron, $\sum\limits_a^N \epsilon_a$. However, this is not correct: if we just add all the eigenvalues together, we end up counting the electron-electron interactions between the electrons twice (see Szabo and Ostlund, *Modern quantum chemistry* eqns: 3.81-3.82):

$$\sum\limits_a^N \epsilon_a = \sum\limits_a^N \left<a\left|h\right|a\right> + \sum\limits_a^N \sum\limits_b^N \left[ \left<aa\left|\right.bb\right> - \left<ab\left|\right.ba\right>\right] $$
$$E_0 = \sum\limits_a^N \left<a\left|h\right|a\right> + \frac{1}{2} \sum\limits_a^N  \sum\limits_b^N \left[ \left<aa\left|\right.bb\right> - \left<ab\left|\right.ba\right>\right] $$
Above, $h$ is the core Hamiltonian and $a$ and $b$ correspond to the occupied orbitals. 



Let's have a look at the molecular orbitals and their energies. We will be using the Aufbau-principle to populate the molecular orbitals: populate the orbitals from the lowest orbital upwards. Since we are using the Numpy 'eigh', the eigenvalues are already ordered from the lowest eigenvalue to the highest.

In [56]:
maxiter = 50
conv_tol = 1e-8
iterstep = 0
conv = 1e0

print(f"{'Iteration':>10} {'Total Energy':>15} {'Dmat Difference':>18}")
while iterstep < maxiter:
    # Update the two-electron part
    Jmat = np.einsum('kl, ijkl->ij', Dmat, Vee, optimize=True) # Compute the Coulomb integrals
    Kmat = np.einsum('kl, ikjl->ij', Dmat, Vee, optimize=True) # Compute the exchange integrals

    Fmat = Hcore + 2*Jmat - Kmat # Construct the full Fock matrix
    #e_elec = np.trace(np.dot(2*Hcore + 2*Jmat - Kmat, Dmat)) # Compute the electronic energy
    e_elec = np.sum(Dmat * (Hcore + Fmat)) # Compute the electronic energy
    Dmat_old = Dmat.copy() # Store old density matrix

    Fmat_ = np.dot(X.T, Fmat.dot(X)) # Transform the Fockian from the AO to the orthonormal MO basis
    mo_eigs, mo_vecs = np.linalg.eigh(Fmat_) # Solve the Fockian
    Cvec = np.dot(X, mo_vecs) # Transform the MO coefficients from MO basis to AO basis

    # Construct a new density matrix
    Dmat *= 0
    for a in range(nelec//2):
        Dmat += np.einsum('i,j->ij', Cvec[:, a], Cvec[:, a].T)
        
    conv = np.linalg.norm(Dmat-Dmat_old) # Evaluate difference in the density matrices
    iterstep += 1
    print(f"{iterstep:>10} {e_elec:>15.10f} {conv:>18.3e}") # Print out energy
    if conv < conv_tol: # Check convergence. If converged, end cycle
        break 

 Iteration    Total Energy    Dmat Difference
         1   -1.7873257912          1.533e-01
         2   -1.8361123242          2.198e-02
         3   -1.8371958862          3.175e-03
         4   -1.8372185962          4.576e-04
         5   -1.8372190684          6.595e-05
         6   -1.8372190782          9.502e-06
         7   -1.8372190784          1.369e-06
         8   -1.8372190784          1.973e-07
         9   -1.8372190784          2.843e-08
        10   -1.8372190784          4.096e-09


Next, we'll construct the density matrix for our initial guess from the molecular orbitals.